<a href="https://colab.research.google.com/github/sznajder/Notebooks/blob/master/GPU_HowRunTFJobsOnGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo do Davide sobre como rodar um job de TF na GPU 

## Setup the environment

In [ ]:
# General packages
import os, glob, re
import psutil
import GPUtil

import math
import numpy as np
from numpy.random import default_rng
import scipy as sp
import scipy.interpolate
import matplotlib
import pandas as pd
import dask.dataframe as dd
matplotlib.rcParams['figure.figsize'] = [10, 10] 
import matplotlib.pyplot as plt
# %matplotlib widget
# %matplotlib inline
import csv
import time

# ML border recognition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Only needed for animations:
import matplotlib.animation as anim
from matplotlib import rc
rc('animation', html='jshtml')

# Only needed to get a progress bar
from tqdm import tqdm

import seaborn as sns

# Tensorflow related packages
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.metrics import MeanRelativeError
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l1, l2, l1_l2

from sklearn.model_selection import train_test_split
import locale


## Load the Data

In [ ]:

folder='.'

## !! only if saved!! load x_train, y_train, x_valid, y_valid from fast access
x_train = np.load('x_train.npz')['arr_0']
y_train = np.load('y_train.npz')['arr_0']
x_valid = np.load('x_valid.npz')['arr_0']
y_valid = np.load('y_valid.npz')['arr_0']

print('Train shapes: x = ', x_train.shape, ' , y = ', y_train.shape)
print('Valid shapes: x = ', x_valid.shape, ' , y = ', y_valid.shape)


# load stats from train_stats.npz, which saved as np.savez_compressed('train_stats.npz', mean, std)
mean = np.load('train_stats.npz')['arr_0']
std = np.load('train_stats.npz')['arr_1']
print(mean)
print(std)



## Get GPU information

In [ ]:
# Set ID number of the GPU to be used
gpu_to_use = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = gpu_to_use
#os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/home/ddicroce/miniconda3/pkgs/cuda-nvcc-11.4.152-0"
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/home/ddicroce/miniconda3/envs/tf-py38"

K.clear_session()
print("TF version", tf.__version__)

# Print CPU and memory information
print(f"CPU count: {psutil.cpu_count()}")  # Number of CPU cores
print(f"Memory: {psutil.virtual_memory().total / (1024 ** 3):.2f} GB")  # Total system memory in GB

# Print GPU information
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = GPUtil.getGPUs()
for i, gpu in enumerate(gpus):
    print(f"GPU {i}: {gpu.name}, Memory: {gpu.memoryTotal / 1024:.2f} GB")



## Create and Train the Model

In [ ]:
dropout         = 1
loss            = "categorical_crossentropy"
last_activation = "sigmoid"

def create_DFF3LC(dim):
  model = Sequential()
  model.add(Dense(4096, input_dim=dim, activation="relu"))
  model.add(Dropout(5/100))
  model.add(BatchNormalization())
  model.add(Dense(2048, activation="relu"))
  model.add(Dropout(5/100))
  model.add(Dense(1024, activation="relu"))
  model.add(Dropout(5/100))
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(5/100))
  model.add(Dense(256, activation="relu"))
  model.add(Dropout(5/100))
  model.add(Dense(128, activation="relu"))
  model.add(Dropout(5/100))
  model.add(BatchNormalization())
  model.add(Dense(32, activation="relu"))
  model.add(Dense(3, activation=last_activation))
  return model

modelname = 'DFF3LSC_4096_2048_1024_512_256_128_32_do%s_%s_%s'%(dropout, loss, last_activation)
model = create_DFF3LC(x_train.shape[1])
#model = load_model('%s/checkpoint_%s_part1'%(folder,modelname))
reduce_lr        = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)
early_stopping   = EarlyStopping(monitor='val_loss', patience=20)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='%s/checkpoint_%s_part1'%(folder,modelname), save_weights_only=False, monitor='val_loss', mode='m
in', save_best_only=True)
def lr_schedule(epoch):
    warmup_epochs  = 10
    initial_lr     = 0.005
    if epoch < warmup_epochs:
        return initial_lr * (warmup_epochs - epoch)
    else:
        return initial_lr
def set_lr(epoch):
    return lr_schedule(epoch)

def on_reduce_lr(epoch, logs):
    model_checkpoint.on_epoch_end(epoch, logs)
    model.load_weights('%s/checkpoint_%s_part1'%(folder,modelname))
lr_scheduler = LambdaCallback(on_epoch_begin=lambda epoch,logs: 
                                           setattr(model.optimizer, "learning_rate", set_lr(epoch)))
opt = Nadam(learning_rate=0.005, epsilon=1e-07)
model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])

with tf.device('/GPU:%s'%(gpu_to_use)):
    history = model.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=200, batch_size=1024,callbacks=[reduce_lr,lr_scheduler,early_stopping,m
odel_checkpoint])
    #history = model.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid),initial_epoch=100, epochs=250, batch_size=1024,callbacks=[reduce_lr,lr_schedul
er,early_stopping,model_checkpoint])

np.save('%s/history_%s_part1.npy'%(folder,modelname), history.history)
